**YOUR NAMES HERE**

Spring 2026

CS 443: Bio-Inspired Machine Learning

Project 1: Hebbian Learning

#### Week 1: TensorFlow, Datasets, and Custom Neural Network Library

In [ ]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

plt.style.use(['seaborn-v0_8-colorblind', 'seaborn-v0_8-darkgrid'])
plt.rcParams.update({'font.size': 18})

np.set_printoptions(suppress=True, precision=4)

# Automatically reload your external source code
%load_ext autoreload
%autoreload 2

## Task 2: Implement familiar neural network layers: Dense, Dropout, Flatten

Now that the CIFAR10 and MNIST data loading and preprocessing pipeline is ready, implement and test the layers that will be used in this project and subsequent ones throughout the semester.

There are many methods to implement here, but they are mostly small — either one or just several lines of code. *You implemented all these methods in CS343 in NumPy so it should be helpful to have your CS343 CNN project open. The new element here is writing them in TensorFlow rather than NumPy.*

**NOTE:**
- For reasons that will become clear very quickly, it is **critical** to have your code run fast and on the GPU. This means **you must write 100% TensorFlow code** in `layers.py`. If you use any NumPy your neural network may not work or run too slowly to do anything useful! *To help ensure this, do NOT import NumPy in `layers.py`.*
- Ignore the methods that have (Week 3) or (Project 2) in the docstrings for now. You will implement those in a few weeks.

In [ ]:
from layers import Layer

### 2a. Implement and test `Layer` class methods

These methods are in the `Layer` class in `layers.py`:
- Constructor
- Get/set methods: `get_name`, `get_act_fun_name`, `get_prev_layer_or_block`, `get_wts`, `get_b`, `has_wts`, `set_activation_function`, `get_num_units`, `get_mode`, `set_mode`
- `compute_net_activation(net_in)`
- `__call__(x)`. Does the forward pass thru the layer. Uses the functional API.

#### Test: Basic `Layer` methods

**NOTE:** You should never instantiate `Layer` objects, but we do so here only to test your code.

In [ ]:
# Test hidden_0
hidden_0 = Layer('Dense_0', activation='linear', prev_layer_or_block=None)
print(f'The layer is called {hidden_0.get_name()} and has {hidden_0.get_act_fun_name()} activation.')
print('You should see:')
print('The layer is called Dense_0 and has linear activation.\nThe previous layer is None')
print(f'The previous layer is {hidden_0.get_prev_layer_or_block()} and should be None.')
print(f'The layer is training?\n{hidden_0.get_mode()}. You should see')
print("<tf.Variable 'Variable:0' shape=() dtype=bool, numpy=False>")
print(f'Does the layer have wts? {hidden_0.has_wts()}. It should be False.')
print('Setting the network to training mode.')
hidden_0.set_mode(True)
print(f'The layer is training?\n{hidden_0.get_mode()}. You should see')
print("<tf.Variable 'Variable:0' shape=() dtype=bool, numpy=True>")


In [ ]:
# Test hidden_1
hidden_1 = Layer('Dense_1', activation='relu', prev_layer_or_block=hidden_0)
print(f'The layer is called {hidden_1.get_name()} and has {hidden_1.get_act_fun_name()} activation.')
print('You should see:')
print('The layer is called Dense_1 and has relu activation.')
print(f'The previous layer is {hidden_1.get_prev_layer_or_block().get_name()} and should be Dense_0.')

#### Test: computing net activation

In [ ]:
tf.random.set_seed(0)
x_test_input = tf.random.uniform(shape=(2, 3), minval=-2, maxval=2)
print('Your activations from Dense_0:')
test_acts = hidden_0.compute_net_activation(x_test_input)
print(test_acts.numpy())
print('they should be:')
print('''[[-0.8321 -1.1737  0.1416]
 [ 0.245  -0.3333  1.2313]]''')
print('Your activations from Dense_1:')
hidden_1.set_activation_function('relu')
test_acts = hidden_1.compute_net_activation(x_test_input)
print(test_acts.numpy())
print('they should be:')
print('''[[0.     0.     0.1416]
 [0.245  0.     1.2313]]''')
print('Your activations from Dense_1 (after changing act fun to softmax):')
hidden_1.set_activation_function('softmax')
test_acts = hidden_1.compute_net_activation(x_test_input)
print(test_acts.numpy())
print('they should be:')
print('''[[0.2295 0.163  0.6075]
 [0.2357 0.1322 0.6321]]''')

### 2b. Implement and test `Dense` layer class methods

These methods are in the `Dense` class in `layers.py`:
- Constructor
- `has_wts`
- `init_params(input_shape)`
- `compute_net_input(x)`.

In [ ]:
from layers import Dense

#### Test: Dense layer basics

In [ ]:
tf.random.set_seed(1)
# x_test_input2 = tf.random.uniform(shape=(2, 7), minval=-2, maxval=2)
layer_0 = Layer('Layer_0', activation='linear', prev_layer_or_block=None)
hidden_2 = Dense('Dense_2', 7, activation='linear', prev_layer_or_block=layer_0, wt_scale=1e-1)
print(f'Dense layer is called {hidden_2.get_name()} and has previous layer {hidden_2.get_prev_layer_or_block().get_name()}')
print('You should see:')
print('Dense layer is called Dense_2 and has previous layer Layer_0')
print(f'Does the layer have weights? {hidden_2.has_wts()}. It should be True.')

print('----------Initializing wts and biases Test1/2----------')
hidden_2.init_params(input_shape=(2, 3))
print(f'Your wts:\n{hidden_2.get_wts().numpy()} They should be:')
print('''[[-0.1101  0.1546  0.0384 -0.088  -0.1225 -0.0981  0.0088]
 [-0.0203 -0.0558 -0.0721 -0.0626 -0.0715 -0.0348 -0.0336]
 [ 0.0183  0.1109  0.128  -0.0021 -0.032   0.0373  0.0253]]''')
print(f'Your biases:\n{hidden_2.get_b().numpy()} They should be:')
print('''[0. 0. 0. 0. 0. 0. 0.]''')
print('----------Initializing wts and biases Test2/2----------')
hidden_2.init_params(input_shape=(4, 5, 2, 3))
print(f'Your wts:\n{hidden_2.get_wts().numpy()} They should be:')
print('''[[ 0.0403 -0.1088 -0.0063  0.1337  0.0712 -0.0489 -0.0764]
 [-0.1037 -0.1252  0.0021 -0.0551 -0.1743 -0.0335 -0.1043]
 [ 0.1009  0.1236 -0.0684  0.0674 -0.0421 -0.1041 -0.068 ]]''')
print(f'Your biases:\n{hidden_2.get_b().numpy()} They should be:')
print('''[0. 0. 0. 0. 0. 0. 0.]''')

#### Test: Dense layer forward pass 1/2

This test calls the rest of your `Dense` layer methods via your`__call__` implementation in `Layer`.

In [ ]:
tf.random.set_seed(1)
x_test_input2 = tf.random.uniform(shape=(4, 3), minval=-2, maxval=2)
print('Your activations from Dense_2 w/ linear:')
hidden_2.set_activation_function('linear')
test_acts = hidden_2(x_test_input2)
print(test_acts.numpy())
print('they should be:')
print('''[[-0.1677  0.0095 -0.024  -0.2323 -0.3973 -0.0429 -0.1007]
 [ 0.1333  0.2031 -0.0391  0.0493  0.1025 -0.0186  0.068 ]
 [ 0.1683 -0.0954 -0.0563  0.3118  0.1538 -0.1511 -0.1619]
 [ 0.0064 -0.0575 -0.0328  0.0568 -0.0847 -0.0865 -0.1202]]''')
print('Your activations from Dense_2 w/ relu:')
hidden_2.set_activation_function('relu')
test_acts = hidden_2(x_test_input2)
print(test_acts.numpy())
print('they should be:')
print('''[[0.     0.0095 0.     0.     0.     0.     0.    ]
 [0.1333 0.2031 0.     0.0493 0.1025 0.     0.068 ]
 [0.1683 0.     0.     0.3118 0.1538 0.     0.    ]
 [0.0064 0.     0.     0.0568 0.     0.     0.    ]]''')

#### Test: Dense layer forward pass 2/2

This tests lazy initialization.

In [ ]:
tf.random.set_seed(1)
x_test_input2 = tf.random.uniform(shape=(4, 3), minval=-2, maxval=2)
print('Your activations from Dense_2 w/ linear:')
for i in range(5):
    test_acts = hidden_2(x_test_input2)
print(test_acts.numpy())
print('they should be:')
print('''[[0.     0.0095 0.     0.     0.     0.     0.    ]
 [0.1333 0.2031 0.     0.0493 0.1025 0.     0.068 ]
 [0.1683 0.     0.     0.3118 0.1538 0.     0.    ]
 [0.0064 0.     0.     0.0568 0.     0.     0.    ]]''')

#### Test: `__str__` and output shape

In [ ]:
print(hidden_2)
print('The above should print:')
print('Dense layer output(Dense_2) shape: [4, 7]')

### 2c. Implement and test `Dropout` layer class methods

Please complete the `Dropout` class then test your implementation below.

**Note:** If you did not learn dropout layers in CS343, please see the supplementary video on the notes website. Please come talk to me if any questions.

In [ ]:
from layers import Dropout

#### Test: `Dropout` layer

In [ ]:
tf.random.set_seed(0)
print('-------------------------Test1/2-------------------------')
x_test_input_drop = tf.random.normal(shape=(4, 5))
dummy_layer = Layer('Blarg', activation='linear', prev_layer_or_block=None)
drop = Dropout('Dropout_Layer', rate=0.7, prev_layer_or_block=dummy_layer)
drop.set_mode(True)
drop_net_in = drop.compute_net_input(x_test_input_drop)
print(f'The layer preceding your dropout layer is {drop.get_prev_layer_or_block().get_name()}. It should be Blarg.')
print(f'Your Dropout net_in is:\n{drop_net_in}')
print('It should be EITHER:')
print('''[[ 0.      1.4097 -0.     -0.     -0.    ]
 [ 0.     -0.0466  0.      0.      0.    ]
 [-0.     -0.      2.6454 -0.     -3.1994]
 [-0.     -1.2027 -0.      1.0128  1.7384]]''')
print('or:')
print('''[[ 0.      0.     -1.399  -3.4535 -0.    ]
 [ 0.     -0.      3.9629  0.      0.    ]
 [-2.3524 -0.      0.     -2.325  -0.    ]
 [-3.0023 -0.     -0.      0.      0.    ]]''')
print('-------------------------Test2/2-------------------------')
drop.set_mode(False)
drop_net_act = drop(x_test_input_drop)
print(f'Your Dropout net_act is:\n{drop_net_act}')
print('It should be:')
print('''[[ 1.5111  0.4229 -0.4197 -1.036  -1.2368]
 [ 0.4703 -0.014   1.1889  0.6025  0.5997]
 [-0.7057 -0.433   0.7936 -0.6975 -0.9598]
 [-0.9007 -0.3608 -0.2238  0.3038  0.5215]]''')

#### Test: `Dropout` layer `__str__`

In [ ]:
print(drop)
print('The above should print:')
print('Dropout layer output(Dropout_Layer) shape: [4, 5]')

### 2d. Implement and test `Flatten` layer class methods

In [ ]:
from layers import Flatten

#### Test: `Flatten` forward pass

In [ ]:
tf.random.set_seed(0)
x_test_input_flat = tf.random.normal(shape=(2, 1, 2, 3))
flat = Flatten('pancake', prev_layer_or_block=None)
flat_acts = flat(x_test_input_flat)
print(f'Your Flatten layer net_act is:\n{flat_acts}')
print('It should be:')
print('''[[ 1.5111  0.4229 -0.4197 -1.036  -1.2368  0.4703]
 [-0.014   1.1889  0.6025  0.5997 -0.7057 -0.433 ]]''')

#### Test: `Flatten` layer `__str__`

In [ ]:
print(flat)
print('The above should print:')
print('Flatten layer output(pancake) shape: [2, 6]')

## Task 3: Build the `DeepNetwork` and `LinearDecoder` classes

With the layers implemented, now let's tackle the network itself. This is the last step before we can start training on some data!

We will divide up the job of creating a deep network into a parent and child classes:
- `DeepNetwork` class (located in `network.py`): Serves as parent class for numerous neural networks (*both artificial and bio-inspired*) that we develop with this semester.
- `LinearDecoder` class (located in `decoder_nets.py`): This is just a simple single-layer artificial neural network configured with softmax activation (i.e. a softmax output layer).

The division of labor between the parent `DeepNetwork` class and specific network instances (e.g. `LinearDecoder`) will allow us to rapidly build many neural networks with minimal added code. All the "boilerplate" code that every network needs like the `fit` method, will reside in the parent and be reused automatically without massive amounts of error-prone copy-pasting!

In [ ]:
from layers import Dense
from network import DeepNetwork

### 3a. Implement `DeepNetwork` part 1/2

To help see the big picture, start implementing and testing only the following methods in `DeepNetwork`:

- `DeepNetwork` constructor
- `compile(loss, lr, print_summary))`: Just add the optimizer to the existing implementation.

#### Test: Constructor and `compile`

This code that will be used for most forthcoming tests.

In [ ]:
def create_test_net():
    # Build fake layers for testing
    test_layer1 = Dense('dense1', 3, prev_layer_or_block=None)
    test_layer2 = Dense('dense2', 4, prev_layer_or_block=test_layer1)
    test_layer3 = Dropout('dropout1', rate=0, prev_layer_or_block=test_layer2)
    test_layer_out = Dense('dense_out', 5, prev_layer_or_block=test_layer3)
    # Build fake net for testing
    test_net = DeepNetwork(input_feats_shape=(2,))
    test_net.test_layer1 = test_layer1
    test_net.test_layer2 = test_layer2
    test_net.test_layer3 = test_layer3
    test_net.output_layer = test_layer_out

    def __call__(self, x):
        net_act = self.test_layer1(x)
        net_act = self.test_layer2(net_act)
        net_act = self.test_layer3(net_act)
        net_act = self.output_layer(net_act)
        return net_act

    setattr(DeepNetwork, '__call__', __call__)
    return test_net

test_net = create_test_net()
test_net.compile(lr=0.1)

Executing the above cell should print out:


```
---------------------------------------------------------------------------
Dense layer output(dense_out) shape: [1, 5]
Dropout layer output(dropout1) shape: [1, 4]
Dense layer output(dense2) shape: [1, 4]
Dense layer output(dense1) shape: [1, 3]
---------------------------------------------------------------------------
```

In [ ]:
print(f'Input ƒeature shape is {test_net.input_feats_shape} and should be (2,).')
print(f'Optimizer is {test_net.opt.name} and should be adam.')
print(f'Optimizer lr is {float(test_net.opt.learning_rate):.3f} and should be 0.100.')
print(f'Number of parameters discovered in network is {len(test_net.all_net_params)} and should be 6.')

### 3b. Implement `LinearDecoder` network

Recall this is a super simple single-layer softmax network for that will be used for classification.

Since the parent class `DeepNetwork` will handle training, getting predictions, and other tasks, all that needs to be done in the `LinearDecoder` class (in `decoder_nets.py`) is implement:
- constructor: Where you create and configure network layer and assign it to instance variables.
- `__call__(x)`: Performs a forward pass thru your `LinearDecoder` network with data samples `x`. If you `LinearDecoder` network is called `dnet` and the data is called `data`, recall that you would call the `__call__` method like this: `dnet(data)`.

In [ ]:
from decoder_nets import LinearDecoder

#### Test: `LinearDecoder` forward pass shapes and architecture

In [ ]:
test_dnet = LinearDecoder(C=4, input_feats_shape=(32*32*3,))
print('My beautiful linear decoder network!')
test_dnet.compile()
print()
print('You should see:')
print()
print('''My beautiful linear decoder network!
---------------------------------------------------------------------------
Dense layer output(Output Layer) shape: [1, 4]
---------------------------------------------------------------------------''')


The network layer name in the parentheses probably will be different (*your chosen layer names*) and that's ok — it should have no bearing on the functionality of your network.

#### Test: `LinearDecoder` forward pass output

Be cautious about small errors in the activations — any discrepancy may suggest a potential bug in your code.

In [ ]:
tf.random.set_seed(0)
test_x = tf.random.normal(shape=(2, 8*8*3))
test_dnet = LinearDecoder(C=4, input_feats_shape=(8*8*3,))
test_dnet.compile()
test_net_act_out = test_dnet(test_x)
print(f'Your output activations after the forward pass are:\n{test_net_act_out}')
print('Your activations should be:')
print('''[[0.2527 0.252  0.2486 0.2467]
 [0.2486 0.2532 0.2487 0.2496]]''')


### 3c. Implement `DeepNetwork` part 2/2

Now that the `LinearDecoder` network has been built and tested, let's make it functional so that we can train it with some data!

Implement the following methods in `DeepNetwork` to finish up the class:
- `set_layer_training_mode(is_training)`: Configures each net layer to operate in training mode or non-training mode.
- `accuracy(y_true, y_pred)`
- `predict(x, output_layer_net_act)`: Perform the forward pass on data samples `x` and predict their classes.
- `loss(out_net_act, y, eps)`: Compute the general cross-entropy loss. **See equation below.**
- `update_params(tape, loss)`: Perform one "step" of backprop — update the network weights and biases based on gradients recorded on the gradient tape.
- `train_step(x_batch, y_batch)`: Do one "step" of training — forward backward pass.
- `test_step(x_batch, y_batch)`: Do one "step" of testing/prediction.
- `fit(x, y, x_val, y_val, batch_size, max_epochs, val_every, print_every, verbose)`: Train the deep neural network using training and (optionally) validation data.

Except for one case noted in `fit`, **your code should be implemented in 100% TensorFlow** in `DeepNetwork`!

#### General cross-entropy loss

Here is a refresher on the equation for the general cross-entropy loss $L$ with int-coded classes $y_i$ and output layer net acts $z_i$ for sample $i$. You should implement this equation in the `loss` method.

$$
L = -\frac{1}{N} \sum_{i=1}^N Log \left (z_{i, y_i} + \epsilon \right )
$$

The only thing new about the above equation is addition of $\epsilon$, which is a very small fudge factor to prevent possibly taking the log of 0 in rare cases.

**NOTE:** You already implemented this in CS343, so you can adapt your code. But remember, the code should be written in TensorFlow rather than NumPy here.

#### Test: `set_layer_training_mode`

**NOTE:** The following tests go back to using the simpler network test code from Task 3a.

In [ ]:
test_net = create_test_net()
test_net.compile()

test_net.set_layer_training_mode(False)
should_be_false = [test_net.test_layer1.get_mode(),
                   test_net.test_layer2.get_mode(),
                   test_net.test_layer3.get_mode(),
                   test_net.output_layer.get_mode()]

print(f'All net layers should NOT be in training mode. Are they in training mode? {tf.reduce_any(should_be_false)}')

test_net.set_layer_training_mode(True)
should_be_true = [test_net.test_layer1.get_mode(),
                  test_net.test_layer2.get_mode(),
                  test_net.test_layer3.get_mode(),
                  test_net.output_layer.get_mode()]

print(f'All net layers should be in training mode. Are they in training mode? {tf.reduce_any(should_be_true)}')

test_net.set_layer_training_mode(False)
should_be_false = [test_net.test_layer1.get_mode(),
                   test_net.test_layer2.get_mode(),
                   test_net.test_layer3.get_mode(),
                   test_net.output_layer.get_mode()]

print(f'All net layers should NOT be in training mode. Are they in training mode? {tf.reduce_any(should_be_false)}')

#### Test: `accuracy` and `loss`

In [ ]:
tf.random.set_seed(0)
test_y_true = tf.constant([1, 0, 0, 1, 2, 1, 1, 0, 0, 1, 2])
test_y_pred = tf.constant([1, 0, 2, 1, 0, 1, 1, 0, 0, 1, 0])
test_acc = test_net.accuracy(test_y_true, test_y_pred)
print(f'Your test acc is {test_acc:.4f} and it should be 0.7273.')

test_net_acts = tf.random.uniform(shape=(2, 5))
test_y = tf.constant([0, 2])
test_loss = test_net.loss(test_net_acts, test_y, eps=1e-1)
print(f'Your test loss is {test_loss:.4f} and it should be 0.4215.')


#### Test: `update_params` and `train_step`

In [ ]:
tf.random.set_seed(0)  # Make sure everyone's wts/biases are the same
test_net = create_test_net()
test_net.compile(lr=10.0)  # this is an insanely high lr just for testing :)

tf.random.set_seed(1)
test_x = tf.random.uniform(shape=(3, 2))
test_y_true = tf.constant([3, 0, 2])

print(28*'-', 'Before train step', 28*'-')
wts_0 = test_net.test_layer1.wts.numpy()
b_2 = test_net.output_layer.b.numpy()
print(f'1st layer wts:\n{wts_0} and they should be:')
print('''[[ 0.0015  0.0004 -0.0004]
 [-0.001  -0.0012  0.0005]]''')
print(f'Output layer bias:\n{b_2} and they should be:')
print('''[0. 0. 0. 0. 0.]''')

loss = test_net.train_step(test_x, test_y_true)

print(28*'-', 'After train step', 28*'-')
wts_0 = test_net.test_layer1.wts.numpy()
b_2 = test_net.output_layer.b.numpy()
print(f'1st layer wts:\n{wts_0} and they should be:')
print('''[[ 0.0015  0.0004  9.9995]
 [-0.001  -0.0012 10.0004]]''')
print(f'Output layer bias:\n{b_2} and they should be:')
print('''[0.     0.     0.     9.9999 0.    ]''')

print()
print(f'The loss on the test mini-batch is {loss.numpy():.4f} and should be 31.9420')

#### Test: `test_step`

In [ ]:
tf.random.set_seed(0)  # Make sure everyone's wts/biases are the same
test_net = create_test_net()
test_net.compile()

tf.random.set_seed(1)
test_x = tf.random.uniform(shape=(5, 2))
test_y_true = tf.constant([3, 1, 2, 1, 0])

test_acc, test_loss = test_net.test_step(test_x, test_y_true)

print(f'Your acc is {test_acc:.3f} and it should be 0.200')
print(f'Your loss is {test_loss:.3f} and it should be 33.902')


**NOTE:** Your `fit` method will be tested in the next task!